In [1]:
import cv2
import numpy as np
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from keras.applications.vgg16 import VGG16 
from keras.applications.vgg16 import preprocess_input
import os
from moviepy.editor import ImageSequenceClip

2024-05-27 12:43:50.533854: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-27 12:43:51.807402: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Function to extract features from a batch of frames
def extract_features_batch(frames, model):
    # Resize frames and stack them into a single array
    resized_frames = [cv2.resize(frame, (224, 224)) for frame in frames]
    batch_array = np.array(resized_frames)
    
    # Preprocess the batch
    batch_array = preprocess_input(batch_array)
    
    # Extract features using the model
    features = model.predict(batch_array)
    
    # Flatten the features for each frame
    flattened_features = features.reshape(features.shape[0], -1)
    
    return flattened_features

In [3]:
# Load pre-trained VGG16 model
model = VGG16(weights = 'imagenet', include_top = False)

# Initialize KMeans clustering
kmeans = KMeans(n_clusters = 2, random_state = 42)

In [4]:
# Read the video
video_path = "video - MERGE.mp4"
cap = cv2.VideoCapture(video_path)

In [5]:
frames = []
frame_indices = []
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frames.append(frame)
    frame_indices.append(int(cap.get(cv2.CAP_PROP_POS_FRAMES)))

# frames

In [6]:
# Extract features from each frame
batch_size = 32
features = []
# Process frames in batches
for start in range(0, len(frames), batch_size):
    end = min(start + batch_size, len(frames))
    batch_frames = frames[start:end]
    batch_features = extract_features_batch(batch_frames, model)
    features.append(batch_features)

# Convert list of features to a single numpy array
features = np.vstack(features)

1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 

In [7]:
# Reduce dimensionality using PCA
pca = PCA(n_components = 100, random_state = 22)
features_pca = pca.fit_transform(features)

# Cluster the features
kmeans.fit(features_pca)

KMeans(n_clusters=2, random_state=42)

In [8]:
# Group frames into scenes
scenes = {}
for i, label in enumerate(kmeans.labels_):
    if label not in scenes:
        scenes[label] = []
    scenes[label].append(frame_indices[i])  # Use frame indices instead of index

In [9]:
# Create output directory
output_dir = 'video_clips'
os.makedirs(output_dir, exist_ok=True)

# Save frames from each scene into separate video clips
fps = cap.get(cv2.CAP_PROP_FPS)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

In [10]:
for scene_id in sorted(scenes.keys()):
    print(f"Saving Scene {scene_id + 1}...")
    frames = []

    for index in scenes[scene_id]:
        cap.set(cv2.CAP_PROP_POS_FRAMES, index - 1)  # Set the frame position
        ret, frame = cap.read()
        if not ret:
            print(f"Warning: Frame at index {index} could not be read.")
            break
        frames.append(frame)

    # print(frames, fps)
    if frames:
        output_path = os.path.join(output_dir, f'scene_{scene_id + 1}.mp4')
        clip = ImageSequenceClip(frames, fps=fps)

        # print(clip)
        try:
            clip.write_videofile(output_path, codec='libx264')
        except Exception as e:
            print(f"An error occurred while saving the video: {e}")

cap.release()
print("Video clips saved successfully.")

Saving Scene 1...
Moviepy - Building video video_clips/scene_1.mp4.
Moviepy - Writing video video_clips/scene_1.mp4



Moviepy - Done !
Moviepy - video ready video_clips/scene_1.mp4
Saving Scene 2...
Moviepy - Building video video_clips/scene_2.mp4.
Moviepy - Writing video video_clips/scene_2.mp4



t:  54%|█████▍    | 819/1523 [00:22<00:17, 40.64it/s, now=None]